In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

from tqdm.notebook import tqdm
tqdm.pandas()
from geoid.censusnames import stusab
import rowgenerators as rg
from geoid.acs import Puma
from pathlib import Path

from demosearch import FileCache


%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
cache = FileCache(Path(pkg.path).parent.joinpath('data', 'cache'))

pkg

# Tract Income Distributions
`civicknowledge.com-income_dist-tract-1.2.2` Last Update: 2021-04-02T16:31:01

_Estimated portions of households per tracts in income ranges_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [tract_income_dist](notebooks/Extract.ipynb#tract_income_dist)**. Full tract income distribution estimates
* ** [income_quartiles](notebooks/Extract.ipynb#income_quartiles)**. Estimated portions of households per tracts in quartile ranges
* ** [census](notebooks/Extract.ipynb#census)**. Census variables for income

## References
<ul><li> <strong>pums</string>, <em>pums:{st}/h/2019/5</em>. PUMS template</li><li> <strong>puma_tract_map</string>, <em>data/puma_tract_map.csv</em>. Puma to tract map</li><li> <strong>us_tracts</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_tracts</em>. US Tracts, with tract numbers</li><li> <strong>households</string>, <em>census://2019/5/US/tract/B11001</em>. US Households by tract</li><li> <strong>median_income</string>, <em>census://2019/5/US/tract/B19013</em>. US Household median income by tract</li><li> <strong>agg_by_quintile</string>, <em>census://2019/5/US/tract/B19082</em>. Shares of Aggregate Household Income by Quintile</li><li> <strong>agg_hh_income</string>, <em>census://2019/5/US/tract/B19025</em>. Aggregate Household Income</li><li> <strong>agg_income</string>, <em>census://2019/5/US/tract/B19313</em>. Aggregate  Income</li><ul>

In [3]:
ptm = pkg.reference('puma_tract_map').dataframe()
ptm = ptm.rename(columns={'tract':'geoid','puma':'PUMA'})

In [4]:
%%time

k = 'pums'

if cache.exists(k):
    pums = cache.get(k)
    
else:
    frames = [rg.dataframe(pkg.reference('pums').url.format(st=st), low_memory=False) for st in tqdm(list(stusab.values()))]
    pums = pd.concat(frames)
    pums = pums[['ST','PUMA','HINCP', 'WGTP']]
    pums['PUMA'] = pums.progress_apply(lambda r: str(Puma(r.ST, r.PUMA)), axis=1)
    cache.put(k, pums)

CPU times: user 866 ms, sys: 614 ms, total: 1.48 s
Wall time: 1.6 s


In [5]:
%%time
cf = {
    'households':'hh',
    'median_income':'mi',
    'agg_by_quintile':'abq',
    'agg_income':'ai',
    'agg_hh_income':'ahhi',
    
}
census = {}


for resource_name, file_name in cf.items():

    if  not cache.exists(file_name):
        print("Loading ", resource_name)
        df = pkg.reference(resource_name).dataframe() 
        
        # Must convert from CensusDataFrame to DatFrame to avoid:
        # TypeError: can't pickle _thread.RLock objects
        cache.put_df(file_name, pd.DataFrame(df.reset_index()))
        df = cache.get_df(file_name)
            
    else:
        print("Reading ", resource_name)
        df = cache.get_df(file_name)
        
    census[resource_name] = df

mi = census['median_income']
hh = census['households']
agg = census['agg_income']
agg_hh = census['agg_hh_income']
abq = census['agg_by_quintile']


Reading  households
Reading  median_income
Reading  agg_by_quintile
Reading  agg_income
Reading  agg_hh_income
CPU times: user 214 ms, sys: 101 ms, total: 315 ms
Wall time: 335 ms


In [6]:
col_map = {
 'b11001_001': 'households',
 'b19013_001': 'median_income',
 'b19313_001': 'agg_income',
 'b19025_001': 'agg_hh_income',
 'b19082_001': 'hhinc_00',
 'b19082_002': 'hhinc_20',
 'b19082_003': 'hhinc_40',
 'b19082_004': 'hhinc_60',
 'b19082_005': 'hhinc_80',
 'b19082_006': 'hhinc_95'

}

t = hh.merge(mi, on='geoid').merge(agg, on='geoid').merge(agg_hh, on='geoid').merge(abq, on='geoid')

census = t[ ['geoid']+list(col_map.keys())].rename(columns=col_map)



In [7]:
# Convert the percentages of total income that each quantile has to the
# income in that quantile. 
quant_cols = [c for c in census if c.startswith('hhinc')]
for c in quant_cols:
    census[c] = pd.to_numeric(census[c])
census.loc[:,quant_cols] = (census.loc[:,quant_cols]/100).multiply(census.agg_hh_income, axis=0)

In [8]:
# This may not be the correct weighting -- maybe
# The weights are only vild within the PUMA?
samp = pums.dropna()
samp = samp.sample(int(10e6), replace=True, weights=samp.WGTP)

In [9]:
step = 5_000

# Clip so we don't deal with crazy extremes
samp['HINCP'] = samp.HINCP.clip(-step, 500_000)

# Quantize the median incomes of each puma. This becomes the index we will use
# to match tracts to PUMA distributions
samp['medinc'] = samp.groupby('PUMA').HINCP.transform(lambda g: (g.median()/step).round()*step).astype(int)
samp.head()

,ST,PUMA,HINCP,WGTP,medinc
36335,16,79500US1600100,50300.0,58,45000
84167,55,79500US5501000,121000.0,5,60000
72703,25,79500US2500503,20000.0,19,110000
12185,33,79500US3301000,61000.0,13,90000
80427,37,79500US3703600,80000.0,22,45000


In [10]:
# Build the list of bin boundaries
inc_bins = np.arange(-step, samp.HINCP.max()+step, step)

# Assign the household incomes to bins
samp['inc_bin'] = pd.cut(samp.HINCP, inc_bins).apply(lambda e: e.left)
samp.head()

,ST,PUMA,HINCP,WGTP,medinc,inc_bin
36335,16,79500US1600100,50300.0,58,45000,50000.0
84167,55,79500US5501000,121000.0,5,60000,120000.0
72703,25,79500US2500503,20000.0,19,110000,15000.0
12185,33,79500US3301000,61000.0,13,90000,60000.0
80427,37,79500US3703600,80000.0,22,45000,75000.0


In [11]:
# Group by the median income index ( and across pums ) and count up the number of people

medinc_bins = samp.groupby('medinc').inc_bin.value_counts().unstack().fillna(0)
medinc_bins = medinc_bins.divide(samp.groupby('medinc').inc_bin.count(), axis=0)
medinc_bins.columns = list(medinc_bins.columns)
medinc_bins.sort_index(level=['medinc', ]).head(10)

,-5000.0,0.0,5000.0,10000.0,15000.0,20000.0,25000.0,30000.0,35000.0,40000.0,...,450000.0,455000.0,460000.0,465000.0,470000.0,475000.0,480000.0,485000.0,490000.0,495000.0
medinc,,,,,,,,,,,,,,,,,,,,,
15000,0.087147,0.108049,0.151156,0.137419,0.110372,0.078622,0.063838,0.049875,0.042002,0.031069,...,0.000000,0.000000,0.000000,0.000000,0.000028,0.000000,0.000000,0.000000,0.000000,0.000142
20000,0.063115,0.092174,0.128728,0.121183,0.102957,0.085623,0.062401,0.053964,0.050930,0.036248,...,0.000076,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000127,0.000000,0.000408
25000,0.047904,0.063358,0.113072,0.109452,0.094291,0.076596,0.066255,0.058448,0.056117,0.043197,...,0.000000,0.000000,0.000000,0.000045,0.000091,0.000023,0.000000,0.000000,0.000000,0.000543
30000,0.038057,0.042893,0.098949,0.094247,0.085745,0.074983,0.069389,0.055281,0.054098,0.042083,...,0.000163,0.000035,0.000122,0.000192,0.000122,0.000157,0.000140,0.000029,0.000087,0.001614
35000,0.031513,0.035261,0.075935,0.083662,0.078445,0.070203,0.066696,0.057742,0.055545,0.045546,...,0.000089,0.000158,0.000106,0.000117,0.000119,0.000078,0.000097,0.000075,0.000153,0.001966
40000,0.022013,0.028329,0.057544,0.073642,0.071284,0.068237,0.065467,0.057620,0.058413,0.047983,...,0.000143,0.000178,0.000154,0.000122,0.000146,0.000186,0.000095,0.000127,0.000154,0.002363
45000,0.017832,0.025188,0.046742,0.062825,0.064180,0.062887,0.061588,0.055207,0.056568,0.049267,...,0.000234,0.000141,0.000249,0.000242,0.000177,0.000210,0.000224,0.000138,0.000184,0.003028
50000,0.014733,0.022304,0.039802,0.055043,0.055918,0.056926,0.056990,0.052254,0.054325,0.047007,...,0.000229,0.000227,0.000228,0.000236,0.000220,0.000207,0.000248,0.000192,0.000180,0.003730
55000,0.012796,0.018925,0.032668,0.047311,0.050905,0.051811,0.052554,0.048443,0.051955,0.045658,...,0.000330,0.000243,0.000261,0.000267,0.000259,0.000299,0.000264,0.000210,0.000203,0.004381


In [12]:

mi_max = max(medinc_bins.index)
mi_min = min(medinc_bins.index)

In [13]:
census['medinc'] = (census.median_income/step).clip(mi_min, mi_max).round().fillna(0).astype(int)
census.head()

,geoid,households,median_income,agg_income,agg_hh_income,hhinc_00,hhinc_20,hhinc_40,hhinc_60,hhinc_80,hhinc_95,medinc
0,14000US01001020100,709,60208.0,62231000,58707900,1702529.1,4091940.63,8471549.97,13491075.42,30956675.67,13256243.82,15000
1,14000US01001020200,688,43958.0,40718100,36164700,1348943.31,3149945.37,6025039.02,9203916.15,16436856.15,5265580.32,15000
2,14000US01001020300,1360,55345.0,86089300,82071900,3373155.09,8338505.04,14280510.6,20091201.12,35988528.15,11514687.57,15000
3,14000US01001020400,1675,59663.0,125932600,123080000,6363236.0,14264972.0,19988192.0,29403812.0,53059788.0,20172812.0,15000
4,14000US01001020500,4483,66108.0,414151800,403905900,12924988.8,41319573.57,62645805.09,88980469.77,198035062.77,108933421.23,15000


In [14]:
t = census.reset_index().merge(medinc_bins.reset_index(), on='medinc')

t.loc[:,-5000:]= t.loc[:,-5000:].multiply(t.households, axis=0)

tracts = pkg.reference('us_tracts').dataframe()
tracts = tracts[['geoid','tract_id']]
tract_income_dist = tracts.merge(t)#.drop(columns=['medinc','households'])
tract_income_dist = tract_income_dist.rename(columns={ c:str(int(c//1000))+'k' for c in tract_income_dist.loc[:,-5000:].columns })
tract_income_dist.head()

,geoid,tract_id,index,households,median_income,agg_income,agg_hh_income,hhinc_00,hhinc_20,hhinc_40,...,450k,455k,460k,465k,470k,475k,480k,485k,490k,495k
0,14000US01055010700,0,318,1376,46078.0,81490500,79543200,4422601.92,8948610.0,12901907.04,...,0.0,0.0,0.0,0.0,0.038971,0.0,0.0,0.0,0.0,0.194857
1,14000US01055001300,1,306,1092,22946.0,42149100,35495200,1508546.0,3247810.8,4976427.04,...,0.0,0.0,0.0,0.0,0.030928,0.0,0.0,0.0,0.0,0.154639
2,14000US01055000900,2,302,898,30238.0,43570000,34296100,1755960.32,3549646.35,5219866.42,...,0.0,0.0,0.0,0.0,0.025433,0.0,0.0,0.0,0.0,0.127167
3,14000US01055001700,3,308,684,24462.0,29897100,25409600,1384823.2,2492681.76,3402345.44,...,0.0,0.0,0.0,0.0,0.019372,0.0,0.0,0.0,0.0,0.096862
4,14000US01055010501,4,314,3200,73248.0,294057000,277982800,11897663.84,29215992.28,46728908.68,...,0.0,0.0,0.0,0.0,0.090631,0.0,0.0,0.0,0.0,0.453155


In [15]:
income_ranges = tract_income_dist[['geoid']].copy()
income_ranges['lt25k'] = tract_income_dist.loc[:,'-5k':'20k'].sum(axis=1)
income_ranges['25k_50k'] = tract_income_dist.loc[:,'25k':'45k'].sum(axis=1)
income_ranges['50k_75k'] = tract_income_dist.loc[:,'50k':'70k'].sum(axis=1)
income_ranges['75k_120k'] = tract_income_dist.loc[:,'75k':'115k'].sum(axis=1)
income_ranges['gt120k'] = tract_income_dist.loc[:,'120k':].sum(axis=1)
income_ranges['gt50k'] = tract_income_dist.loc[:,'50k':].sum(axis=1)
income_ranges['gt60k'] = tract_income_dist.loc[:,'60k':].sum(axis=1)
income_quartiles = income_ranges
income_quartiles.describe()


,lt25k,25k_50k,50k_75k,75k_120k,gt120k,gt50k,gt60k
count,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000,72913.000000
mean,1125.080695,361.007201,111.162936,50.774038,24.297651,186.234625,127.977100
std,548.979484,176.152295,54.241595,24.775028,11.855960,90.872583,62.446012
min,8.073185,2.590461,0.797666,0.364337,0.174351,1.336354,0.918319
25%,754.169990,241.992240,74.515322,34.035120,16.287329,124.837770,85.786281
50%,1046.822930,335.896454,103.430724,47.242325,22.607568,173.280616,119.075337
75%,1398.679223,448.797383,138.195678,63.121332,30.206384,231.523394,159.098731
max,14268.680922,4578.424097,1409.808627,643.934746,308.151609,2361.894981,1623.051943


In [16]:
income_quartiles.head()

,geoid,lt25k,25k_50k,50k_75k,75k_120k,gt120k,gt50k,gt60k
0,14000US01055010700,925.725161,297.039538,91.465730,41.777274,19.992296,153.235301,105.300555
1,14000US01055001300,734.659794,235.731959,72.587629,33.154639,15.865979,121.608247,83.567010
2,14000US01055000900,604.143310,193.852838,59.692024,27.264529,13.047298,100.003852,68.720856
3,14000US01055001700,460.171519,147.656282,45.466976,20.767192,9.938031,76.172199,52.344171
4,14000US01055010501,2152.849213,690.789623,212.711000,97.156452,46.493712,356.361165,244.885012


In [17]:
income_ranges

,geoid,lt25k,25k_50k,50k_75k,75k_120k,gt120k,gt50k,gt60k
0,14000US01055010700,925.725161,297.039538,91.465730,41.777274,19.992296,153.235301,105.300555
1,14000US01055001300,734.659794,235.731959,72.587629,33.154639,15.865979,121.608247,83.567010
2,14000US01055000900,604.143310,193.852838,59.692024,27.264529,13.047298,100.003852,68.720856
3,14000US01055001700,460.171519,147.656282,45.466976,20.767192,9.938031,76.172199,52.344171
4,14000US01055010501,2152.849213,690.789623,212.711000,97.156452,46.493712,356.361165,244.885012
...,...,...,...,...,...,...,...,...
72908,14000US72005400900,712.458536,228.608191,70.394047,32.152713,15.386513,117.933273,81.041634
72909,14000US72127008900,752.151694,241.344624,74.315906,33.944035,16.243741,124.503682,85.556701
72910,14000US72127009000,526.775292,169.027586,52.047723,23.772969,11.376430,87.197122,59.920301
72911,14000US72119130702,1038.076980,333.090121,102.566585,46.847627,22.418687,171.832899,118.080492


In [18]:
tract_income_dist

,geoid,tract_id,index,households,median_income,agg_income,agg_hh_income,hhinc_00,hhinc_20,hhinc_40,...,450k,455k,460k,465k,470k,475k,480k,485k,490k,495k
0,14000US01055010700,0,318,1376,46078.0,81490500,79543200,4422601.92,8948610.0,12901907.04,...,0.0,0.0,0.0,0.0,0.038971,0.0,0.0,0.0,0.0,0.194857
1,14000US01055001300,1,306,1092,22946.0,42149100,35495200,1508546.0,3247810.8,4976427.04,...,0.0,0.0,0.0,0.0,0.030928,0.0,0.0,0.0,0.0,0.154639
2,14000US01055000900,2,302,898,30238.0,43570000,34296100,1755960.32,3549646.35,5219866.42,...,0.0,0.0,0.0,0.0,0.025433,0.0,0.0,0.0,0.0,0.127167
3,14000US01055001700,3,308,684,24462.0,29897100,25409600,1384823.2,2492681.76,3402345.44,...,0.0,0.0,0.0,0.0,0.019372,0.0,0.0,0.0,0.0,0.096862
4,14000US01055010501,4,314,3200,73248.0,294057000,277982800,11897663.84,29215992.28,46728908.68,...,0.0,0.0,0.0,0.0,0.090631,0.0,0.0,0.0,0.0,0.453155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72908,14000US72005400900,73996,73080,1059,9180.0,15483000,14956600,10469.62,822613.0,1974271.2,...,0.0,0.0,0.0,0.0,0.029993,0.0,0.0,0.0,0.0,0.149966
72909,14000US72127008900,73997,73825,1118,11151.0,21579200,21189400,50854.56,1184487.46,2515181.78,...,0.0,0.0,0.0,0.0,0.031664,0.0,0.0,0.0,0.0,0.158321
72910,14000US72127009000,73998,73826,783,12044.0,14271200,13088100,90307.89,918784.62,1904318.55,...,0.0,0.0,0.0,0.0,0.022176,0.0,0.0,0.0,0.0,0.110881
72911,14000US72119130702,73999,73710,1543,21911.0,54237600,48334500,1285697.7,4205101.5,6713662.05,...,0.0,0.0,0.0,0.0,0.043701,0.0,0.0,0.0,0.0,0.218506
